In [184]:
import pandas as pd
import numpy as np 
import re 
import os

os.chdir(r'C:\R workspace\r_examples\texts\CE\data')

In [24]:
# load db
base1 = pd.read_csv("./M1.csv", sep=",", encoding="windows-1251")
base2 = pd.read_csv("./2nd_M1.csv", sep=",", encoding="windows-1251")
base3 = pd.read_csv("./4th_M1.csv", sep=",", encoding="windows-1251")

In [25]:
# merge db
base1.columns = "db1_"+base1.columns
base2.columns = "db2_"+base2.columns
base3.columns = "db3_"+base3.columns
db = pd.concat([base1, base2, base3], axis=0)
db.fillna(" ", inplace=True);

In [139]:
# load pairs and stopwords

pairs = {}
with open('./stopwords.txt', 'r') as st:
    stopwords = st.read().split('\n')   

with open('./pairs_raw.txt', 'r') as pr:
    tmp = pr.read().split('\n\n')

for tmp_pair in tmp:
    tmp_pair = tmp_pair.split('\n')
    for tmp_token in tmp_pair:
        if tmp_token != tmp_pair[0]:
            pairs[tmp_token] = tmp_pair[0]
            

## Отдельные вопросы

In [285]:
# slice by qt 

TARGET_QT = 'c115'
qt_list = pd.Series(db.columns)
db_qt = db.loc[:, list(qt_list.apply(lambda x: TARGET_QT in x))]

In [286]:
# rename qt with tokens

def prepare_token(token):
    token = token.lower()
    token = token.strip("(){}<>-–,. :;!?``“”‘’«»\"")
    # pairs 
    keys = pairs.keys()
    if token in keys:
        token = pairs[token]
    return token

word_columns = []
mask_columns = []
for col in db_qt.columns:
    levels = db_qt[col].value_counts().index
    if np.sum(levels != ' '): 
        word = levels[levels != ' '][0]
        word = prepare_token(word)
        if len(word) > 2 and word not in stopwords: # drop stopwords   
            word_columns.append(word)
            mask_columns.append(True)        
        else:
            mask_columns.append(False)
    else:
        mask_columns.append(False)

db_qt = db_qt.loc[:, mask_columns]
db_qt.columns = word_columns

In [287]:
# binarize to int 0/1

def binarize(col):  
    return pd.Series(col != ' ', dtype=int)

db_qt = db_qt.apply(binarize, axis=1)

In [288]:
# tokens slider 
  
for col in np.unique(db_qt.columns):
    if type(db_qt[col]) != pd.Series:
        tmp_series = db_qt[col].sum(1)
        db_qt.drop(col, axis=1, inplace=True)
        db_qt[col] = (tmp_series > 0).astype(int)


In [289]:
# save 
csv_name = TARGET_QT + ".csv"
db_qt.to_csv(csv_name, sep=',', index=False)